In [20]:
import requests as rq
import bs4 as bs
import re
import time

In [15]:
url = f"https://www.atlasobscura.com/places?page=202&sort=likes_count"
raw = rq.get(url)
soup = bs.BeautifulSoup(raw.content)

#ciao

In [18]:
raw.status_code == 200

True

In [9]:
len(soup.find_all("a",attrs={'data-place-id':re.compile(r"\d*")}))

0

In [ ]:
cities_url = []
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
for page in range(1,401):
    try:
        url = f"https://www.atlasobscura.com/places?page={page}&sort=likes_count"
        raw = rq.get(url,headers=headers)
        if raw.status_code != 200:
            print("Sleeping")
            time.sleep(120)
            raw = rq.get(url,headers=headers)
        
        soup = bs.BeautifulSoup(raw.content)
        
        for d in soup.find_all("a",attrs={'data-place-id':re.compile(r"\d+")}):
            cities_url.append(d["href"])
            
    except Exception as e:
        print(page)
        print(e)
        print(raw.status_code)

In [26]:
len(cities_url)

7200

In [28]:
with open("links.txt","w") as f:
    for link in cities_url:
        complete_url = "https://www.atlasobscura.com"+link+"\n"
        f.write(complete_url)

In [19]:
url = f"https://www.atlasobscura.com/places?page=2&sort=likes_count"

In [20]:
raw = rq.get(url)
soup = bs.BeautifulSoup(raw.content)

In [22]:
with open("pretty.txt","w") as f:
    f.write(soup.prettify())